In [1]:
# Check out the configurations of Sentinel Hub

#!sentinelhub.config --show

In [23]:
# Update the configurations with the client id, client secret id and instance id from Sentinel Hub

#!sentinelhub.config --sh_client_id 'a8e69011-6acf-4803-b9ef-bcb00ed47df3' --sh_client_secret 'm0<l..i&~i_;dz|?C0rC[]JI_5aIy3DL[HEtYCmz' --instance_id '1b8e6d88-ff0b-4a08-9a75-cca01ace6cf1'
#config.save()

In [31]:
# Create a config instance from Sentinel Hub

#from sentinelhub import SHConfig
#config = SHConfig()
#config.instance_id = '1b8e6d88-ff0b-4a08-9a75-cca01ace6cf1'
#config.sh_client_id = 'a8e69011-6acf-4803-b9ef-bcb00ed47df3'
#config.sh_client_secret = 'm0<l..i&~i_;dz|?C0rC[]JI_5aIy3DL[HEtYCmz'

In [2]:
from sentinelhub import SHConfig
config = SHConfig()
config

SHConfig(
  instance_id='37abf98d-a04e-40df-9a9b-6b58fb18ee6e',
  sh_client_id='8a419e16-5053-4203-b22c-4e2c448547c1',
  sh_client_secret='m#khV[sw1X8O5tYGwQ&)w?F439(Y#t:hv+ehvje}',
  sh_base_url='https://services.sentinel-hub.com',
  sh_auth_base_url='https://services.sentinel-hub.com',
  geopedia_wms_url='https://service.geopedia.world',
  geopedia_rest_url='https://www.geopedia.world/rest',
  aws_access_key_id='',
  aws_secret_access_key='',
  aws_session_token='',
  aws_metadata_url='https://roda.sentinel-hub.com',
  aws_s3_l1c_bucket='sentinel-s2-l1c',
  aws_s3_l2a_bucket='sentinel-s2-l2a',
  opensearch_url='http://opensearch.sentinel-hub.com/resto/api/collections/Sentinel2',
  max_wfs_records_per_query=100,
  max_opensearch_records_per_query=500,
  max_download_attempts=4,
  download_sleep_time=5.0,
  download_timeout_seconds=120.0,
  number_of_download_processes=1
)

In [5]:
# Logging config

import logging, sys

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

fhandler = logging.FileHandler(filename='api_log.log', mode='a')
strhandler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.addHandler(strhandler)

In [6]:
# IPython magic functions

%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [7]:
# Import packages

import datetime, os, sys

import matplotlib.pyplot as plt
import numpy as np
from shapely import geometry
import pandas as pd
import datetime as dt
from datetime import datetime
import time, ast
from dateutil.relativedelta import relativedelta

from sentinelhub import (
    CRS,
    BBox,
    Geometry,
    DataCollection,
    DownloadRequest,
    MimeType,
    MosaickingOrder,
    SentinelHubDownloadClient,
    SentinelHubRequest,
    bbox_to_dimensions,
    get_image_dimension
)

In [8]:
# Import CSV file with data about the Area of Interest (AoI)

try:
    aoi_csv = pd.read_csv('../aoi_TI.csv', header=0)
    logger.info('CSV data imported')
    print(aoi_csv.head())
except Exception as e:
    logger.error('Error importing the csv file: ', e)
    

CSV data imported
   Area of interest End of fire  Bbox coordinates  \
0      200308132301  2003-09-05               NaN   
1      201104262301  2011-05-18               NaN   
2      199703152101  1997-03-24               NaN   
3      199704152103  1997-04-21               NaN   
4      200604032101  2006-04-04               NaN   

                                 Polygon coordinates            Location  \
0  [[[615898.47949999943375587,129344.17940000072...                Leuk   
1  [[[635402.48279999941587448,126972.39050000160...                Visp   
2  [[[698436.13769999891519547,110118.48950000107...  Ronco_sopra_Ascona   
3  [[[711619.76649999991059303,108634.33590000122...            Magadino   
4  [[[711919.04329999908804893,115556.63140000030...            Cugnasco   

     x_coor    y_coor         CRS  Longitude   Latitude Unnamed: 10  \
0  615400.0  129800.0  EPSG:21781   7.638587  46.319424   EPSG:4326   
1  635375.0  126975.0  EPSG:21781   7.897712  46.293259   EPSG:4

INFO:__main__:CSV data imported


In [9]:
# Parse the coordinates of boxes
def parse_coord_bbox(str_coordinates):
    try:
        coordinates = str_coordinates[1:-1].split(',')
        coord_list = [float(coord) for coord in coordinates]
        return coord_list
    except Exception as e:
        logger.error('Error occured parsing the box coordinates: ', e)

In [10]:
# Parse the coordinates of polygones
def parse_coord_poly(str_coordinates):
    try:
        coord_str = str_coordinates[1:-1].replace(' ','')
        coord_list = ast.literal_eval(coord_str)
        return coord_list
    except Exception as e:
        logger.error('Error occured parsing the poly coordinates: ', e)

In [11]:
#Create a bbox from the polygone coordinates
# +5000
from math import floor, ceil
def create_bbox(poly_coordinates):
    parsed_coord = parse_coord_poly(poly_coordinates)
    long, lat = zip(*[(coord[0], coord[1]) for coord in parsed_coord])
    bbox_coords = [min(long)-500, min(lat)-500, max(long)+500, max(lat)+500]
    return BBox(bbox=bbox_coords, crs='21781').transform(CRS.WGS84)


In [15]:
# Retrieve satellite images providing polygons as AoI delimitations
# mosaicking_order=MosaickingOrder.LEAST_CC,
# mosaicking_order=MosaickingOrder.LEAST_RECENT,
# maxcc=0.3

try:
    aoi_poly = aoi_csv.loc[aoi_csv['End of fire'].notna()]
    aoi_poly = aoi_csv[pd.to_datetime(aoi_csv['End of fire']).dt.year >= 2013]
    aoi_poly.reset_index(inplace=True)
    start = time.time()
    

    for index, row in aoi_poly.iterrows():
        time_from = row['End of fire']
        date_delta = datetime.strptime(row['End of fire'], '%Y-%m-%d').date() + relativedelta(weeks=+2)
        time_until = date_delta.strftime('%Y-%m-%d')
        aoi_name = row['Location']
        folder_name = str(row['Area of interest'])
        aoi_polygon = Geometry(geometry.Polygon(parse_coord_poly(row['Polygon coordinates'])), '21781').transform(CRS.WGS84)
        aoi_bbox_coord = create_bbox(row['Polygon coordinates'])
        height = 512
        width = get_image_dimension(aoi_bbox_coord, height=512)
        if width < height:
            width = 512
            height = get_image_dimension(aoi_bbox_coord, width=512)
        
        evalscript_all_bands = """
        //VERSION=3

        function setup() {
        return {
            input: ["B01", "B02", "B03", "B04", "B05", "B06", "B07", "B10", "BQA", "dataMask"],
            output: {
            id: "raw",
            bands: 10,
            sampleType: SampleType.UINT16 //floating point values are automatically rounded to the nearest integer by the service.
            }
        };
        }
        function evaluatePixel(sample, scenes, inputMetadata, customData, outputMetadata) {
        return [sample.B01, sample.B02, sample.B03, sample.B04, sample.B05, sample.B06, sample.B07, sample.B10, sample.BQA, sample.dataMask]
        }
        """
        request_all_bands = SentinelHubRequest(
            evalscript=evalscript_all_bands,
            input_data=[
                SentinelHubRequest.input_data(
                    data_collection=DataCollection.LANDSAT_OT_L2,
                    time_interval=(time_from, time_until),
                    mosaicking_order=MosaickingOrder.LEAST_CC,
                )
            ],
            responses=[SentinelHubRequest.output_response('raw', MimeType.TIFF)],
            bbox=aoi_bbox_coord,
            size=(width, height),
            data_folder=f'../Data/WSL/Landsat_images_level2_2W/{folder_name}',
            config=config
        )
        request_all_bands.save_data()
    end = time.time()
    print(f'Execution time: {end - start:.2f} seconds')
except Exception as e:
    print('Error: ', e)
    logger.error('Error occured while requesting satellite images: ', e)
    

Execution time: 8.85 seconds


In [17]:
# Retrieve satellite images providing polygons as AoI delimitations
# mosaicking_order=MosaickingOrder.LEAST_CC,
# mosaicking_order=MosaickingOrder.LEAST_RECENT,
# maxcc=0.3

try:
    aoi_poly = aoi_csv.loc[aoi_csv['End of fire'].notna()]
    aoi_poly = aoi_csv[pd.to_datetime(aoi_csv['End of fire']).dt.year >= 2013]
    aoi_poly.reset_index(inplace=True)
    start = time.time()
    

    for index, row in aoi_poly.iterrows():
        time_from = row['End of fire']
        date_delta = datetime.strptime(row['End of fire'], '%Y-%m-%d').date() + relativedelta(weeks=+2)
        time_until = date_delta.strftime('%Y-%m-%d')
        aoi_name = row['Location']
        folder_name = str(row['Area of interest'])
        aoi_polygon = Geometry(geometry.Polygon(parse_coord_poly(row['Polygon coordinates'])), '21781').transform(CRS.WGS84)
        aoi_bbox_coord = create_bbox(row['Polygon coordinates'])
        height = 512
        width = get_image_dimension(aoi_bbox_coord, height=512)
        if width < height:
            width = 512
            height = get_image_dimension(aoi_bbox_coord, width=512)
        
        evalscript_all_bands = """
        //VERSION=3

        function setup() {
        return {
            input: ["B01", "B02", "B03", "B04", "B05", "B06", "B07", "B08", "B8A", "B09", "B11", "B12", "dataMask"],
            output: {
            id: "raw",
            bands: 13,
            sampleType: SampleType.UINT16 //floating point values are automatically rounded to the nearest integer by the service.
            }
        };
        }
        function evaluatePixel(sample, scenes, inputMetadata, customData, outputMetadata) {
        return [sample.B01, sample.B02, sample.B03, sample.B04, sample.B05, sample.B06, sample.B07, sample.B08, sample.B8A, sample.B09, sample.B11, sample.B12, sample.dataMask]
        }
        """
        request_all_bands = SentinelHubRequest(
            evalscript=evalscript_all_bands,
            input_data=[
                SentinelHubRequest.input_data(
                    data_collection=DataCollection.SENTINEL2_L2A,
                    time_interval=(time_from, time_until),
                    mosaicking_order=MosaickingOrder.LEAST_CC,
                )
            ],
            responses=[SentinelHubRequest.output_response('raw', MimeType.TIFF)],
            bbox=aoi_bbox_coord,
            size=(width, height),
            data_folder=f'../Data/WSL/Sentinel_images_2W/{folder_name}',
            config=config
        )
        request_all_bands.save_data()
    end = time.time()
    print(f'Execution time: {end - start:.2f} seconds')
except Exception as e:
    print('Error: ', e)
    logger.error('Error occured while requesting satellite images: ', e)
    

Execution time: 12.54 seconds
